In [137]:
import os
import sys
import openpyxl
import pygal
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics import mosaicplot
from statsmodels.graphics.mosaicplot import mosaic
sns.set(style="ticks", color_codes=True)
from rpy2.robjects.packages import importr
import requests
import xlrd
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

In [138]:
#  Doctorate Recipients by DemographDoctrate reciepients by Citizenship,  Race/Ethnicity, Sex, S&E Fields, Broad Fields, Detailed Fields, and Year
# Data Download from NCSES Interactive Data Tool
df1 = pd.read_csv('C:/Users/Muluken/WorkingFiles/NSF_NCSES/Data/SED_Export_2020-06-17T23_22_18.852Z.csv', skiprows=[0], header=[1])
df1 = df1.set_index(['S&E Fields', 'Broad Fields', 'Detailed Fields'])
df1.head(3)
#df1.tail()
#df1.dtypes

2018  \
S&E Fields              Broad Fields  Detailed Fields                                      
Science and engineering Total         Total                                        42227   
                        Life sciences Total                                        12780   
                                      Agricultural sciences and natural resources   1445   

                                                                                    2017  \
S&E Fields              Broad Fields  Detailed Fields                                      
Science and engineering Total         Total                                        41294   
                        Life sciences Total                                        12555   
                                      Agricultural sciences and natural resources   1494   

                                                                                    2016  \
S&E Fields              Broad Fields  Detailed Fields                                      
Science and engineering Total         Total                                        41234   
                        Life sciences Total                                        12536   
                                      Agricultural sciences and natural resources   1378   

                                                                                    2015  \
S&E Fields              Broad Fields  Detailed Fields                                      
Science and engineering Total         Total                                        41178   
                        Life sciences Total                                        12493   
                                      Agricultural sciences and natural resources   1434   

                                                                                    2014  \
S&E Fields              Broad Fields  Detailed Fields                                      
Science and engineering Total         Total                                        40633   
                        Life sciences Total                                        12484   
                                      Agricultural sciences and natural resources   1338   

                                                                                    2013  \
S&E Fields              Broad Fields  Detailed Fields                                      
Science and engineering Total         Total                                        39031   
                        Life sciences Total                                        12207   
                                      Agricultural sciences and natural resources   1324   

                                                                                    2012  \
S&E Fields              Broad Fields  Detailed Fields                                      
Science and engineering Total         Total                                        37846   
                        Life sciences Total                                        11964   
                                      Agricultural sciences and natural resources   1255   

                                                                                    2011  \
S&E Fields              Broad Fields  Detailed Fields                                      
Science and engineering Total         Total                                        36332   
                        Life sciences Total                                        11535   
                                      Agricultural sciences and natural resources   1206   

                                                                                    2010  \
S&E Fields              Broad Fields  Detailed Fields                                      
Science and engineering Total         Total                                        34997   
                        Life sciences Total                                        11319   
                            

In [140]:
df1s = df1.stack()
df1s =df1s.to_frame()
df1s.head(10)
df1s = df1s.reset_index()
df1s.columns = ['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']
# df1s.columns
df1s.head()
# df1s.shape

,S&E Fields,Broad Fields,Detailed Fields,Year,Number
0,Science and engineering,Total,Total,2018,42227
1,Science and engineering,Total,Total,2017,41294
2,Science and engineering,Total,Total,2016,41234
3,Science and engineering,Total,Total,2015,41178
4,Science and engineering,Total,Total,2014,40633


In [141]:
df1s = df1s[df1s['Detailed Fields'] != 'Total']
df1s = df1s.reset_index()
df1s = df1s[['S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']]
#df1s["Number"] = df1s["Number"].astype(str).astype(float)
df1s.head(3)
df1s.tail(3)

# Replace no data values '-' to Nan so that it can be dropped
df1s['Number'].replace('-', np.nan, inplace=True)
df1s = df1s.dropna()
#df1s.describe
#df1s.dtypes

# Converting from object to integer
df1s["Number"] = df1s["Number"].astype(str).astype(int)
df1s.dtypes
df1s.head(3)
df1s.shape
df1s['Detailed Fields'].unique()
df = df1s.groupby('Detailed Fields')['Number'].nunique()
df.count()

36

In [163]:
# Setting an index of three columns to conduct aggregation
SE_NSE = df1s.set_index(['S&E Fields', 'Broad Fields', 'Detailed Fields','Year'])
SE_NSE = SE_NSE.groupby(['S&E Fields','Broad Fields', 'Year'])['Number'].sum()
#SE_NSE=SE_NSE.sum(level='Detailed Fields')
SE_NSE = SE_NSE.to_frame()
SE_NSE = SE_NSE.reset_index()
SE_NSE.head(20)
#SE_NSE.columns
#SE_NSE.shape

,S&E Fields,Broad Fields,Year,Number
0,Non-science and engineering,Education,1958,1491
1,Non-science and engineering,Education,1959,1553
2,Non-science and engineering,Education,1960,1549
3,Non-science and engineering,Education,1961,1679
4,Non-science and engineering,Education,1962,1893
5,Non-science and engineering,Education,1963,2137
6,Non-science and engineering,Education,1964,2351
7,Non-science and engineering,Education,1965,2736
8,Non-science and engineering,Education,1966,3040
9,Non-science and engineering,Education,1967,3481


In [167]:
SE_NSE1= SE_NSE.pivot(index='S&E Fields', columns='Year', values='Number')
#SE_NSE1= SE_NSE.pivot(index='S&E Fields', columns='Year', values='Number')
#SE_NSE1 = SE_NSE1.reset_index()
#SE_NSE2 = SE_NSE1.set_index([ 'Year'])

#SE_NSE1.stack(level='S&E Fields')

#SE_NSE1.stack(level='Year')

#SE_NSE1.columns

ValueError: Index contains duplicate entries, cannot reshape

In [165]:
SE_NSE1.head(2)
#SE_NSE1.columns

,Broad Fields,Year,Number
Non-science and engineering,Education,1958,1491
Non-science and engineering,Education,1959,1553


In [161]:
SE_NSE1 = SE_NSE1[['S&E Fields',  'Year', 'Number']]
#SE_NSE1 = SE_NSE.set_index('S&E Fields')
# SE_NSE1 = SE_NSE1.rename_axis(None)
# SE_NSE1.head(20)
#df = SE_NSE1.set_index('S&E Fields')
dic = SE_NSE1.T.to_dict().values()
print(dic)

KeyError: "['S&E Fields'] not in index"

In [157]:
SE_NSE1 = SE_NSE1.rename_axis(None)
SE_NSE1.columns

Index(['Broad Fields', 'Year', 'Number'], dtype='object')

In [58]:

df = SE_NSE1.set_index('Year')
#print(df.T.to_dict(orient='list'))
print(df)

KeyError: "['Year'] not in index"

In [ ]:
# Doctorate Recipients by Demographic Characteristics ---- Data by Citizenship,  Race/Ethnicity, Sex, S&E Fields, Broad Fields, Detailed Fields, and Year
# Data Download from NCSES Interactive Data Tool
df2 = pd.read_csv('C:/Users/Muluken/WorkingFiles/NSF_NCSES/Data/SED_Export_2020-06-17T23_29_53.896Z.csv', skiprows=[0], header=[1])
df2 = df2.set_index(['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields'])
df2.head()

In [ ]:
df2s = df2.stack()
df2s =df2s.to_frame()

df2s = df2s.reset_index()
df2s.columns = ['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']
df2s.head(10)

In [ ]:
df2s = df2s[df2s['Detailed Fields'] != 'Total']
df2s = df2s.reset_index()
df2s = df2s[['Citizenship','Race and Ethnicity', 'Sex','S&E Fields', 'Broad Fields', 'Detailed Fields', 'Year', 'Number']]
df2s.head()